# Data Mart Reports Test

### This demonstrates that the data model is indeed working properly and that the appropriate queries are possible.

##### Imports

In [28]:
import pandas as pd
import psycopg2
import json
import subprocess

##### DB Connection

In [29]:
file=json.load(open('creds.json'))

conn = psycopg2.connect(database='data_mart',\
                        user=file['username'],\
                        password=file['password'],\
                        host='127.0.0.1',\
                        port= '5432')

cursor = conn.cursor()

##### Schema Test and Reference

In [9]:
#Select all table names in data_mart
sql = """
SELECT table_schema
        , table_name 
FROM 
    information_schema.tables
WHERE 
    table_schema NOT IN ('pg_catalog', 'information_schema')"""
cursor.execute(sql)
cursor.fetchall()
#{t[0]:t[1] for t in cursor.fetchall()}

[('loads', 'tmp_import'),
 ('customer_data', 'customers'),
 ('customer_data', 'address_history'),
 ('order_data', 'order_details'),
 ('order_data', 'orders')]

##### Requirement: The ability to look at the total number of orders shipped to a specific state in a given year


In [15]:
sql ="""
WITH base AS (
    
SELECT
order_id
, state
, CAST(EXTRACT(YEAR FROM order_date) as int) AS year
FROM
customer_data.address_history
INNER JOIN 
order_data.orders
USING (customer_id)

)

SELECT
year
, state
, COUNT(order_id) AS orders
FROM
base
GROUP BY
year
, state
ORDER BY
state
, year
"""

pd.read_sql(sql, conn)



/var/tmp/ipykernel_45513/384042218.py:31: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, conn)


,year,state,orders
0,2014,AK,1
1,2015,AK,2
2,2016,AK,1
3,2016,AZ,1
4,2013,CA,2
5,2014,CA,3
6,2015,CA,3
7,2017,CA,1
8,2015,CO,1
9,2016,CO,1


##### Requirement: The ability to look up all customers current addresses


In [25]:
sql = """
WITH base AS (
    
SELECT
*
FROM
customer_data.customers
INNER JOIN
customer_data.address_history
USING (customer_id)
)

,agg as (
SELECT DISTINCT
first_name
, last_name
, CASE 
    WHEN end_date = MAX(end_date) OVER (PARTITION BY customer_id) 
    THEN CONCAT(street_number,', ',city,', ',state,' ',postal_code) 
    ELSE null
    END AS address
FROM base
ORDER BY
last_name
, first_name
)

SELECT * FROM agg WHERE address IS NOT NULL
"""
pd.read_sql(sql, conn)

/var/tmp/ipykernel_45513/2960507529.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, conn)


,first_name,last_name,address
0,Cammy,Albares,"56 E Morehead St, Laredo, TX 78045"
1,Minna,Amigon,"2371 Jerrold Ave, Kulpsville, PA 19443"
2,Stephaine,Barfield,"47154 Whipple Ave Nw, Gardena, CA 90247"
3,Kallie,Blackwood,"701 S Harrison Rd, San Francisco, CA 94104"
4,Brock,Bolognia,"422 E 21st St, Syracuse, NY 13214"
5,Emerson,Bowley,"762 S Main St, Madison, WI 53711"
6,Jina,Briddick,"32860 Sierra Rd, Miami, FL 33133"
7,James,Butt,"2409 Alabama Rd, Riverside, CA 92501"
8,Ezekiel,Chui,"4646 Kaahumanu St, Hackensack, NJ 7601"
9,Delisa,Crupi,"47565 W Grand Ave, Newark, NJ 7105"


##### Summary reports that will show all orders and their order status

In [61]:
#Summary reports that will show all orders and their order status
print("No status included in sample data provided")

No status included in sample data provided


##### Requirement: A way to see what customers did not have order in certain years
###### Quote: "Marketing is really interested in evaluating customer order trends"

In [47]:
#A way to see what customers did not have order in certain years
# ■ Marketing is really interested in evaluating customer order trends

#Strangely worded... I will attempt to find what they did not order by year.

sql = """
WITH base AS (
    
SELECT *, CAST(EXTRACT(year from order_date) as int) as year FROM order_data.orders
INNER JOIN
order_data.order_details
USING (order_id)
)

, products AS (
    
SELECT
DISTINCT
b.product_name
,c.customer_id
,y.year
FROM
base b
CROSS JOIN
(SELECT DISTINCT customer_id FROM base) AS c
CROSS JOIN
(SELECT DISTINCT year FROM base) AS y)

SELECT 
p.product_name, 
customer_id, year, order_date, quantity
FROM products p
LEFT JOIN
base b
USING (product_name,customer_id,year)
WHERE order_date IS NULL
ORDER BY product_name, customer_id, year 
"""

pd.read_sql(sql, con=conn)

/var/tmp/ipykernel_45513/4217004011.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, con=conn)


,product_name,customer_id,year,order_date,quantity
0,Anzamlax,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,2013,None,None
1,Anzamlax,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,2014,None,None
2,Anzamlax,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,2015,None,None
3,Anzamlax,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,2016,None,None
4,Anzamlax,\x01a68ce3aabaaa3fcc8fa58bfd4f84a044947996fd59...,2017,None,None
...,...,...,...,...,...
6266,Zer Dax,\xf8e87f42ce6114735a829d613995c57b9b5251acdc2f...,2013,None,None
6267,Zer Dax,\xf8e87f42ce6114735a829d613995c57b9b5251acdc2f...,2014,None,None
6268,Zer Dax,\xf8e87f42ce6114735a829d613995c57b9b5251acdc2f...,2015,None,None
6269,Zer Dax,\xf8e87f42ce6114735a829d613995c57b9b5251acdc2f...,2016,None,None


##### Requirement Quarterly Sales reports at the product level as well a quarterly report at the customer level
###### Use Case: Product level

In [57]:
#Quarterly Sales reports at the product level as well a quarterly report at the customer level
# Product level
sql = """
WITH base AS (
SELECT 
product_name
,order_id
,price
,quantity
,customer_id
,order_date
,CAST(EXTRACT(QTR FROM order_date) as int) AS quarter
,CAST(EXTRACT(YEAR FROM order_date) as int) AS year
FROM
order_data.order_details
INNER JOIN
order_data.orders
USING (order_id)
)

SELECT 
year
,quarter
,product_name
,sum(quantity) AS quantity
,sum(price) AS revenue
FROM base
GROUP BY
year
,quarter
,product_name

ORDER BY
product_name
,year
,quarter

"""

pd.read_sql(sql, conn)


/var/tmp/ipykernel_45513/2175147423.py:40: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, conn)


,year,quarter,product_name,quantity,revenue
0,2013,3,Anzamlax,3.0,8.99
1,2014,1,Anzamlax,14.0,291.95
2,2014,2,Anzamlax,6.0,92.98
3,2015,2,Anzamlax,1.0,31.99
4,2013,2,Dentodox,1.0,5.99
...,...,...,...,...,...
160,2015,1,Zer Dax,5.0,103.98
161,2015,3,Zer Dax,2.0,170.98
162,2016,1,Zer Dax,4.0,33.98
163,2016,2,Zer Dax,2.0,17.99


##### Requirement Quarterly Sales reports at the product level as well a quarterly report at the customer level
###### Use Case: Product level

In [58]:
#Quarterly Sales reports at the product level as well a quarterly report at the customer level
#Customer level
sql = """
WITH base AS (
SELECT 
product_name
,order_id
,price
,quantity
,customer_id
,order_date
,CAST(EXTRACT(QTR FROM order_date) as int) AS quarter
,CAST(EXTRACT(YEAR FROM order_date) as int) AS year
FROM
order_data.order_details
INNER JOIN
order_data.orders
USING (order_id)
)

SELECT 
year
,quarter
,first_name
,last_name
,customer_id
,sum(quantity) AS quantity
,sum(price) AS revenue
FROM base
INNER JOIN
customer_data.customers
USING (customer_id)
GROUP BY
year
,quarter
,first_name
,last_name
,customer_id
ORDER BY
last_name
,first_name
,year
,quarter
"""

pd.read_sql(sql, conn)


/var/tmp/ipykernel_45513/1125860472.py:46: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(sql, conn)


,year,quarter,first_name,last_name,customer_id,quantity,revenue
0,2015,2,Cammy,Albares,\x7ba38823694b2ad8022642df27d970f814e575365a38...,11.0,128.96
1,2014,1,Minna,Amigon,\x04e39f45829b3256718e228e2c69e2ff10cc4102a6b8...,6.0,95.98
2,2016,2,Minna,Amigon,\x04e39f45829b3256718e228e2c69e2ff10cc4102a6b8...,2.0,45.99
3,2015,3,Stephaine,Barfield,\x7ad02460a8f832e6972a6baee7239524a14a6b97d923...,10.0,156.97
4,2013,4,Kallie,Blackwood,\x9da682d8f85d2aa70b915d8941eef3c32854839a2d4b...,5.0,54.98
...,...,...,...,...,...,...,...
82,2015,1,Tammara,Wardrip,\x607e1d0536b2377fb3675a38c9441a3626328556e28f...,5.0,92.98
83,2015,3,Tammara,Wardrip,\x607e1d0536b2377fb3675a38c9441a3626328556e28f...,1.0,7.99
84,2015,3,Sage,Wieser,\x828533f54460fca012eea44a8724e51c1c5487b05f8d...,5.0,67.98
85,2016,4,Sage,Wieser,\x828533f54460fca012eea44a8724e51c1c5487b05f8d...,4.0,18.99
